# day 7

https://adventofcode.com/7/day/7

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day07.txt')

LOGGER = logging.getLogger('day07')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
def parse_data(s: str):
    equations = []
    for line in s.strip().split('\n'):
        test_val, inputs = line.strip().split(': ')
        test_val = int(test_val)
        inputs = tuple([int(_) for _ in inputs.split(' ')])
        equations.append({'test_val': test_val, 'inputs': inputs})
    return equations

In [ ]:
parse_data(test_data)

In [ ]:
from enum import Enum


class Operators(Enum):
    ADD = 'add'
    MUL = 'mul'

In [ ]:
# sum([2 ** len(d['inputs']) for d in parse_data(load_data())])

In [ ]:
from functools import lru_cache

@lru_cache
def eqn_exists(test_val: int, inputs: tuple[int, ...]) -> bool:
    *head, tail = inputs

    if not head:
        return test_val == tail

    return any([eqn_exists(test_val=test_val / tail, inputs=tuple(head)),
                eqn_exists(test_val=test_val - tail, inputs=tuple(head))])

In [ ]:
assert eqn_exists(test_val=190, inputs=(10, 19))
assert eqn_exists(test_val=3267, inputs=(81, 40, 27))
assert not eqn_exists(test_val=83, inputs=(17, 5))
assert not eqn_exists(test_val=156, inputs=(15, 6))
assert not eqn_exists(test_val=7290, inputs=(6, 8, 6, 15))
assert not eqn_exists(test_val=161011, inputs=(16, 10, 13))
assert not eqn_exists(test_val=192, inputs=(17, 8, 14))
assert not eqn_exists(test_val=21037, inputs=(9, 7, 18, 13))
assert eqn_exists(test_val=292, inputs=(11, 6, 16, 20))

#### function def

In [ ]:
def q_1(data):
    equations = parse_data(data)
    return sum([d['test_val'] for d in equations if eqn_exists(**d)])

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 3749
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
from functools import lru_cache

@lru_cache
def eqn_exists(test_val: int, inputs: tuple[int, ...]) -> bool:
    if test_val <= 0:
        return False

    *head, tail = inputs
    if not head:
        return test_val == tail

    sub_eqs_params = [{'test_val': test_val / tail, 'inputs': tuple(head)},
                      {'test_val': test_val - tail, 'inputs': tuple(head)},]

    # concatenation is possible when tail is the last  n digits in the current test_val
    # AND also only possible when the current value is an exact integer
    int_test_val = int(test_val)
    test_val_is_int = int_test_val == test_val
    if test_val_is_int and str(int_test_val).endswith(str(tail)):
        remainder = str(int_test_val)[:-len(str(tail))]
        if remainder != '':
            remainder = int(remainder)
            sub_eqs_params.append({'test_val': remainder, 'inputs': tuple(head)})

    return any([eqn_exists(**d) for d in sub_eqs_params])

# assert eqn_exists(test_val=7290, inputs=(6, 8, 6, 15))
# assert not eqn_exists(test_val=21037, inputs=(9, 7, 18, 13))
# assert not eqn_exists(test_val=1, inputs=(2, 1))

In [ ]:
assert eqn_exists(test_val=190, inputs=(10, 19))
assert eqn_exists(test_val=3267, inputs=(81, 40, 27))
assert not eqn_exists(test_val=83, inputs=(17, 5))
assert eqn_exists(test_val=156, inputs=(15, 6))
assert eqn_exists(test_val=7290, inputs=(6, 8, 6, 15))
assert not eqn_exists(test_val=161011, inputs=(16, 10, 13))
assert eqn_exists(test_val=192, inputs=(17, 8, 14))
assert not eqn_exists(test_val=21037, inputs=(9, 7, 18, 13))
assert eqn_exists(test_val=292, inputs=(11, 6, 16, 20))
assert not eqn_exists(test_val=1, inputs=(2, 1))

In [ ]:
def q_2(data):
    equations = parse_data(data)
    return sum([d['test_val'] for d in equations if eqn_exists(**d)])

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 11387
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin